In [1]:
import pandas as pd
import pstats

# Path to your .prof file
profiling_file = 'profiling_output.prof'

# Load the profiling data
stats = pstats.Stats(profiling_file)

# Prepare an empty list to hold the data
data = []

# The stats.entries attribute holds the raw profiling data
for func, (cc, nc, tt, ct, callers) in stats.stats.items():
    filename, line_number, function_name = func
    data.append({
        'filename': filename,
        'line_number': line_number,
        'function_name': function_name,
        'call_count': cc,
        'total_time': tt,
        'cumulative_time': ct
    })

# Convert the list of dictionaries into a pandas DataFrame
df = pd.DataFrame(data)

# Optionally, sort the DataFrame by a column of interest, e.g., 'total_time'
df = df.sort_values(by='total_time', ascending=False)

# Show the DataFrame
print(df.head())  # Adjust as necessary to display or analyze the DataFrame

# create a column that skips D:\Documents\.vscode\SecurityCamera\camera if present
df['filename_end'] = df['filename'].apply(lambda x: x.replace('D:\\Documents\\.vscode\\SecurityCamera\\camera', ''))

                                               filename  line_number  \
5566                                                  ~            0   
5705                                                  ~            0   
5496                                                  ~            0   
1487  D:\Documents\.vscode\SecurityCamera\camera\cam...          110   
5557                                                  ~            0   

                                      function_name  call_count  total_time  \
5566  <built-in method _winapi.WaitForSingleObject>           4   46.516674   
5705          <built-in method _imp.create_dynamic>         115    4.019305   
5496                      <built-in method nt.stat>       50050    3.347015   
1487                             _connect_to_camera           1    1.344428   
5557                 <built-in method io.open_code>         832    0.254728   

      cumulative_time  
5566        46.516674  
5705         4.032291  
5496         3.42280

In [5]:
from pathlib import Path


_s = r'''D:\Documents\.vscode\SecurityCamera\camera'''
my_code_df = df[df['filename'].str.startswith(_s)].sort_values(by='cumulative_time', ascending=False)


# my_code_df[['filename_end', 'line_number', 'function_name', 'call_count','cumulative_time']]

# Function to get the source code of the given function from the file.
def get_function_code(filename, function_name, line_number):
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
        # Inspect can help find the code, but here we are making an assumption
        # about the start line of the function (which may include decorators, docstrings, etc.)
        source_code = ''.join(lines[line_number-1:line_number-1+5])  # Adjust the range as needed
        return source_code
    except Exception as e:
        return f"Could not retrieve source code: {e}"



# iterate over filename_end
for filename in my_code_df['filename'].unique():
    # skip if call_count for function is 0
    print(filename)
    file_df = my_code_df[my_code_df['filename'] == filename]
    # filter out call_count = 1
    file_df = file_df[file_df['call_count'] > 1]
    file_df['source_code'] = file_df.apply(
        lambda row: get_function_code(
            filename, 
            row['function_name'], 
            row['line_number'] 
        ), axis=1
    )
    _filename = Path(Path(filename).stem + '.xlsx')
    print(f'Writing to {_filename}')
    file_df[['line_number', 'function_name', 'call_count', 'cumulative_time', 'source_code']].to_excel(_filename)

    

D:\Documents\.vscode\SecurityCamera\camera\filemanager.py
Writing to filemanager.xlsx
D:\Documents\.vscode\SecurityCamera\camera\camera.py
Writing to camera.xlsx
D:\Documents\.vscode\SecurityCamera\camera\MotionDetector.py
Writing to MotionDetector.xlsx
D:\Documents\.vscode\SecurityCamera\camera\kafka_manager.py
Writing to kafka_manager.xlsx
D:\Documents\.vscode\SecurityCamera\camera\frame.py
Writing to frame.xlsx
D:\Documents\.vscode\SecurityCamera\camera\resolution.py
Writing to resolution.xlsx
D:\Documents\.vscode\SecurityCamera\camera\__init__.py
Writing to __init__.xlsx
